In [19]:
from ipynb.fs.full.Notebook1 import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

## Descriptive Data Analysis

- City Profit margins (Scatter plot?)
- Sales per City based on Product ID
- Sales Per Item with Discount and Profit (Bar chart?)
- Products sold per City

In [42]:
profits_sorted = df.groupby(['City'])
profits_sorted

## Inferential Data Analysis

- Products sold depending on the City
- Profit Margin per City
- Sales are dependent on the Location and Day ordered (graph might be a elevation graph)